In [2]:
import os
import torch
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
import torch

In [72]:
class LaneDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_path = r"C:\Users\49179\Desktop\Advance_Lane_Detection_System\archive\TUSimple\train_set", train= True, image_size = (512,256)):
        self._dataset_path = dataset_path
        self._mode = "train" if train else "eval"
        self._image_size = image_size # w, h

        self._data = []

        if self._mode =='train':
            label_files = [os.path.join(self._dataset_path,f"label_data_{suffix}.json") for suffix in ("0313", "0531")]

        elif self._mode =='eval':
            label_files  = [os.path.join(self._dataset_path,f"label_data_{suffix}.json") for suffix in ("0601",)]
        
        for label_file in label_files:
             self._preprocess_labels(label_file)

    def __len__(self):
         return len(self._data)

    def __getitem__(self, idx):
        image_path = os.path.join(self._dataset_path, self._data[idx][0])
        image = cv2.imread(image_path)
        height, width , channels = image.shape
        raw_image = image
        image = cv2.resize(image,self._image_size, interpolation=cv2.INTER_LINEAR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = image[..., None]
        lanes = self._data[idx][1]
        masked_image = np.zeros_like(image)


    def _preprocess_labels(self, labels_path):
            for file in labels_path:
                with open(file, 'r') as f:
                    for lines in f:
                        labels = json.loads(lines)
                        images = labels['raw_file']
                        lanes = labels['lanes']
                        h_samples = labels['h_samples']
                        lane_coordinates = []
                        for lane in lanes:
                             x = np.array([lane]).T
                             y = np.array([h_samples]).T
                             xy = np.hstack((x,y))
                             idx = np.where(xy[:,0]>0)
                             lane_coordinate = xy[idx]
                             lane_coordinates.append(lane_coordinate)
                        self._data.append((images,lane_coordinate))
            return self._data
        
        


In [73]:
x = LaneDataset()
g= x._preprocess_labels(path)

In [96]:
k = g[0][0]
m= r'C:/Users/49179/Desktop/Advance_Lane_Detection_System/archive/TUSimple/train_set/'+k
h = cv2.imread(m)

In [3]:
image = np.zeros((512,256), 'uint8')
cv2.imshow('imag',image)
cv2.waitKey(1)

-1

: 

In [30]:
path = x.hello()

In [24]:
for file in x.hello():
    with open(file, 'r') as f:
        for line in f:
            labels = json.loads(line)
            h_samples = labels['h_samples']

In [36]:
type(h_samples)

list

In [7]:
for x, y,z in os.walk(r"C:\Users\49179\Desktop\Advance_Lane_Detection_System\archive\TUSimple\train_set"):
    print(z)

['label_data_0313.json', 'label_data_0531.json', 'label_data_0601.json', 'readme.md']
[]
[]
['1.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '2.jpg', '20.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']
['1.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '2.jpg', '20.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']
['1.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '2.jpg', '20.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']
['1.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '2.jpg', '20.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']
['1.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '2.jpg', '20.jpg', '3.jpg', '4